<a href="https://colab.research.google.com/github/i-murray/SIGNATEHiroshimaLemon/blob/main/CreateArchieve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Upgrage / Install
!pip install pandas
!pip install h5py
!pip install opencv-python
!pip install numpy

In [32]:
import pandas as pd
import h5py
import cv2
import numpy as np
from collections import defaultdict
import os

In [ ]:
with h5py.File('archive.hdf5', 'w') as hf:

  for split in ['train','test']:

    images = defaultdict(list)
    for _, row in pd.read_csv(f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/{split}_images.csv').iterrows():
      if os.path.exists(f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/{split}_images/'+row['id']):
        im = cv2.imread(f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/{split}_images/'+row['id'])
        im = im[:, :, ::-1].astype(np.float32) / 255
        images[row['class_num']].append(im)

    grp = hf.create_group(split)
    for k in images.keys():
      d = np.stack(images[k],axis=0)
      grp.create_dataset(k,
                      shape=d.shape(),
                      compression=None,
                      chunks=None)